# Deep Learning with Keras

Done by Joel Kambey

Objective: Make use of deep learning model to predict a continuous target value (in this case, the strength of different concretes) and explore optimization options for the model.

## 1. Data Inspection and Preparation

Import Pandas for dataframe, Numpy for array operations.

In [1]:
import pandas as pd
import numpy as np

Download the data and put it in a dataframe.

In [2]:
# use this open-source dataset 
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


#### Overview of the datasets:

The data contains compressive strength of different samples of concrete based on the volumes of the different materials in them (e.g. Cement, Water, etc.). The six materials are displayed in in each columns.

For example, the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

#### Size and Properties of DF

In [3]:
# check the size of the dataframe - (rows, columns)
concrete_data.shape

(1030, 9)

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


There are 1030 samples to train the model.

#### Check for any Null values.

In [5]:
# check for null values in the dataset
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

No adjustment needed for Null values in the dataset.

## 2. Feature Extraction

Split the data into 2 dataframes - features/predictors and target.

Target = Concrete strength.

Features = All the other columns (e.g. materials, age, etc.)

In [6]:
# returns a list of the column names
concrete_data_columns = concrete_data.columns

# set Predictors DF - using the column names as keys (w/o the Target)
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] 

# set Target DF
target = concrete_data['Strength'] 

<a id="item2"></a>

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [9]:
# number of predictors (or columns of X dataset)
n_cols = predictors.shape[1] 

## 3. Building the DL Model

Import Keras and Skicit-Learn Libraries

In [10]:
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [11]:
from keras.models import Sequential
from keras.layers import Dense 

Regression is the most appropriate algorithm to utilize for this case as the target is continuous in nature.

Construct the functions for the Regression Model and MSE (to determine accuracy).

In [12]:
def regression_model_a():
    # create/initialise model object using Keras Sequential() fn
    model = Sequential()
    
    # build the neural network properties
    # set the # of nodes (10), activation fn (ReLu), # of inputs (first hidden layer)
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def get_mse_a(predictors, target, model):  
    # Split the datasets into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)  

    # Train the model on the training data using 50 epochs
    model.fit(X_train, y_train, epochs=50, verbose=2)    
    
    # Evaluate the model on the test data and compute the mean squared error 
    y_hat = model.predict(X_test)  
    
    # compute the mse
    mse = mean_squared_error(y_test, y_hat)
    
    # Return the mean squared error
    return mse

Lets evaluate model 50 times (Repeat runs of split, fit/train, and evaluate MSE/accuracy for each run. This is limited by the runtime and computing power of my system.)

Then store the results in the MSE array.

In [20]:
# construct the model 
model = regression_model_a()

# construct an empty list for the MSEs
mse_list = []

# evaluate the model 50 times
for i in range(50): 
    print(f'This is Epoch run no: {i+1}')
    # the fn get_mse() will split, fit/train, and evaluate the model
    mse_list.append(get_mse_a(predictors, target, model))

mean_mse = np.mean(mse_list)
std_dev_mse = np.std(mse_list)

This is Epoch run no: 1
Epoch 1/50
 - 5s - loss: 50547.5243
Epoch 2/50
 - 0s - loss: 10257.0761
Epoch 3/50
 - 0s - loss: 3752.8235
Epoch 4/50
 - 0s - loss: 3107.5407
Epoch 5/50
 - 0s - loss: 2630.4748
Epoch 6/50
 - 0s - loss: 2200.9166
Epoch 7/50
 - 0s - loss: 1854.2441
Epoch 8/50
 - 0s - loss: 1589.6892
Epoch 9/50
 - 0s - loss: 1425.7324
Epoch 10/50
 - 0s - loss: 1307.8978
Epoch 11/50
 - 0s - loss: 1229.6574
Epoch 12/50
 - 0s - loss: 1162.4356
Epoch 13/50
 - 0s - loss: 1109.9140
Epoch 14/50
 - 0s - loss: 1065.6368
Epoch 15/50
 - 0s - loss: 1020.6599
Epoch 16/50
 - 0s - loss: 982.4206
Epoch 17/50
 - 0s - loss: 946.1291
Epoch 18/50
 - 0s - loss: 913.7136
Epoch 19/50
 - 0s - loss: 883.2436
Epoch 20/50
 - 0s - loss: 858.4498
Epoch 21/50
 - 0s - loss: 830.4483
Epoch 22/50
 - 0s - loss: 804.8560
Epoch 23/50
 - 0s - loss: 779.1418
Epoch 24/50
 - 0s - loss: 757.0460
Epoch 25/50
 - 0s - loss: 733.4843
Epoch 26/50
 - 0s - loss: 710.9376
Epoch 27/50
 - 0s - loss: 689.7965
Epoch 28/50
 - 0s - los

Check the distribution of the accuracies.

In [21]:
print(f'List of the MSEs: {mse_list}')
print(f'Mean: {mean_mse}')
print(f'Standard Deviation: {std_dev_mse}')

List of the MSEs: [380.4960138696942, 166.36852249360217, 122.63561966447703, 112.68938753773901, 112.0021073064842, 110.11014451954293, 105.49922034066401, 105.22810206778362, 109.07105825899993, 102.83994486543922, 78.83966263361826, 52.56866290353435, 51.21229237105666, 50.83159158022278, 50.007257098005965, 56.45016998609075, 53.14213318137437, 53.74354130156248, 50.052829092989086, 49.440305745040895, 51.11994390970107, 52.76623776052097, 51.12548582729204, 51.99554167961591, 48.841115750951275, 48.57753251883636, 49.660183908715325, 56.045234712202735, 51.40909986198888, 48.81166130422452, 48.52893363244116, 57.16085944932073, 51.100642875677615, 49.0997585115938, 52.11905344457291, 49.49373946197573, 50.19638699086193, 49.991729841303176, 53.626795182477494, 55.66442856745486, 50.298696596493535, 49.37755034670622, 49.72643074787209, 49.56875803146006, 48.713424742320846, 48.68161551462516, 52.627653602294544, 78.1812104578036, 54.04272824123433, 51.82203802610612]
Mean: 70.6720

## 4. Model Optimization

### a. Data Normalization

Try normalize the features and check if this improves accuracy.

In [22]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [24]:
# construct the model 
model_b = regression_model_a()

# construct an empty list for the MSEs
mse_list_b = []

# evaluate the model 50 times
for i in range(50): 
    print(f'This is Epoch run no: {i+1}')
    # the fn get_mse() will split, fit/train, and evaluate the model
    mse_list_b.append(get_mse_a(predictors_norm, target, model_b))

mean_mse_b = np.mean(mse_list_b)
std_dev_mse_b = np.std(mse_list_b)

This is Epoch run no: 1
Epoch 1/50
 - 5s - loss: 1607.5301
Epoch 2/50
 - 0s - loss: 1594.7009
Epoch 3/50
 - 0s - loss: 1582.4164
Epoch 4/50
 - 0s - loss: 1570.3531
Epoch 5/50
 - 0s - loss: 1558.3310
Epoch 6/50
 - 0s - loss: 1545.9862
Epoch 7/50
 - 0s - loss: 1533.0805
Epoch 8/50
 - 0s - loss: 1519.3989
Epoch 9/50
 - 0s - loss: 1504.6265
Epoch 10/50
 - 0s - loss: 1488.8475
Epoch 11/50
 - 0s - loss: 1471.7983
Epoch 12/50
 - 0s - loss: 1453.3694
Epoch 13/50
 - 0s - loss: 1433.6792
Epoch 14/50
 - 0s - loss: 1412.6497
Epoch 15/50
 - 0s - loss: 1389.8313
Epoch 16/50
 - 0s - loss: 1365.7866
Epoch 17/50
 - 0s - loss: 1340.2746
Epoch 18/50
 - 0s - loss: 1312.8349
Epoch 19/50
 - 0s - loss: 1284.2342
Epoch 20/50
 - 0s - loss: 1253.5446
Epoch 21/50
 - 0s - loss: 1221.7191
Epoch 22/50
 - 0s - loss: 1187.9193
Epoch 23/50
 - 0s - loss: 1153.4349
Epoch 24/50
 - 0s - loss: 1117.5497
Epoch 25/50
 - 0s - loss: 1080.1759
Epoch 26/50
 - 0s - loss: 1042.3932
Epoch 27/50
 - 0s - loss: 1003.7946
Epoch 28/50
 

In [25]:
print(f'Part A MSEs Mean and Standard Deviation: {mean_mse}, {std_dev_mse}')
print(f'Part B MSEs Mean and Standard Deviation: {mean_mse_b}, {std_dev_mse_b}')

Part A MSEs Mean and Standard Deviation: 70.67206076633134, 51.453265046132685
Part B MSEs Mean and Standard Deviation: 58.523274906805824, 42.89759888951768


From the results, the normalized data produced a more accurate model as the mean MSE is smaller than without normalization.

### b. Increase # of Epoch 

Try to double the numbers of training cycles (Epoch) in the model. This will results in significantly longer runtime however.

In [26]:
def get_mse_c(predictors, target, model):  
    # Split the datasets into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)  

    # Train the model on the training data using 50 epochs
    model.fit(X_train, y_train, epochs=100, verbose=2)    
    
    # Evaluate the model on the test data and compute the mean squared error 
    y_hat = model.predict(X_test)  
    
    # compute the mse
    mse = mean_squared_error(y_test, y_hat)
    
    # Return the mean squared error
    return mse

In [27]:
# construct the model 
model_c = regression_model_a()

# construct an empty list for the MSEs
mse_list_c = []

# evaluate the model 50 times
for i in range(50): 
    print(f'This is Epoch run no: {i+1}')
    # the fn get_mse() will split, fit/train, and evaluate the model
    mse_list_c.append(get_mse_c(predictors_norm, target, model_c))

mean_mse_c = np.mean(mse_list_c)
std_dev_mse_c = np.std(mse_list_c)

This is Epoch run no: 1
Epoch 1/100
 - 5s - loss: 1561.5107
Epoch 2/100
 - 0s - loss: 1545.9887
Epoch 3/100
 - 0s - loss: 1530.2797
Epoch 4/100
 - 0s - loss: 1514.1263
Epoch 5/100
 - 0s - loss: 1497.2691
Epoch 6/100
 - 0s - loss: 1479.7104
Epoch 7/100
 - 0s - loss: 1460.8849
Epoch 8/100
 - 1s - loss: 1440.7476
Epoch 9/100
 - 0s - loss: 1419.5835
Epoch 10/100
 - 0s - loss: 1396.6898
Epoch 11/100
 - 0s - loss: 1372.4308
Epoch 12/100
 - 0s - loss: 1346.7028
Epoch 13/100
 - 0s - loss: 1319.6797
Epoch 14/100
 - 0s - loss: 1291.0492
Epoch 15/100
 - 0s - loss: 1261.0504
Epoch 16/100
 - 0s - loss: 1229.8690
Epoch 17/100
 - 0s - loss: 1197.5603
Epoch 18/100
 - 0s - loss: 1164.3483
Epoch 19/100
 - 0s - loss: 1130.2802
Epoch 20/100
 - 0s - loss: 1095.2116
Epoch 21/100
 - 0s - loss: 1059.5477
Epoch 22/100
 - 0s - loss: 1023.4624
Epoch 23/100
 - 0s - loss: 987.4543
Epoch 24/100
 - 0s - loss: 950.2773
Epoch 25/100
 - 0s - loss: 913.6045
Epoch 26/100
 - 0s - loss: 876.8178
Epoch 27/100
 - 0s - loss: 

In [28]:
print(f'Part A MSEs Mean and Standard Deviation: {mean_mse}, {std_dev_mse}')
print(f'Part B MSEs Mean and Standard Deviation: {mean_mse_b}, {std_dev_mse_b}')
print(f'Part C MSEs Mean and Standard Deviation: {mean_mse_c}, {std_dev_mse_c}')

Part A MSEs Mean and Standard Deviation: 70.67206076633134, 51.453265046132685
Part B MSEs Mean and Standard Deviation: 58.523274906805824, 42.89759888951768
Part C MSEs Mean and Standard Deviation: 72.98822520902064, 12.879615696163482


From the results, we can see with double the iterations (50 to 100), the model has a much lower MSE standard deviation as compared to before

### c. Increase the # of Hidden Layers in the Neural Network to 3.

Try increasing the hidden layers to 3 (10 Nodes each, Act Fn: ReLu). Again, this will result i longer runtime.

Set the Regression Model with 3 hidden layers

In [30]:
def regression_model_d():
    # create/initialise model object using Keras Sequential() fn
    model = Sequential()
    
    # set the # of nodes (10), activation fn (ReLu), # of inputs (first hidden layer)
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))         
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [32]:
# construct the model 
model_d = regression_model_d()

# construct an empty list for the MSEs
mse_list_d = []

# evaluate the model 50 times
for i in range(50): 
    print(f'This is Epoch run no: {i+1}')
    # the fn get_mse() will split, fit/train, and evaluate the model
    mse_list_d.append(get_mse_a(predictors_norm, target, model_d))

mean_mse_d = np.mean(mse_list_d)
std_dev_mse_d = np.std(mse_list_d)

This is Epoch run no: 1
Epoch 1/50
 - 6s - loss: 1556.2094
Epoch 2/50
 - 0s - loss: 1522.3784
Epoch 3/50
 - 0s - loss: 1469.3729
Epoch 4/50
 - 0s - loss: 1382.5510
Epoch 5/50
 - 0s - loss: 1255.1774
Epoch 6/50
 - 0s - loss: 1078.3228
Epoch 7/50
 - 0s - loss: 861.2137
Epoch 8/50
 - 0s - loss: 631.0390
Epoch 9/50
 - 0s - loss: 450.2039
Epoch 10/50
 - 0s - loss: 347.7581
Epoch 11/50
 - 0s - loss: 297.3048
Epoch 12/50
 - 0s - loss: 269.6388
Epoch 13/50
 - 0s - loss: 249.8144
Epoch 14/50
 - 0s - loss: 235.8216
Epoch 15/50
 - 0s - loss: 225.0549
Epoch 16/50
 - 0s - loss: 216.5648
Epoch 17/50
 - 0s - loss: 209.5207
Epoch 18/50
 - 0s - loss: 203.4764
Epoch 19/50
 - 0s - loss: 197.8078
Epoch 20/50
 - 0s - loss: 193.1026
Epoch 21/50
 - 0s - loss: 188.9501
Epoch 22/50
 - 0s - loss: 185.9042
Epoch 23/50
 - 0s - loss: 182.5553
Epoch 24/50
 - 0s - loss: 179.0075
Epoch 25/50
 - 0s - loss: 175.9362
Epoch 26/50
 - 0s - loss: 173.1856
Epoch 27/50
 - 0s - loss: 170.6091
Epoch 28/50
 - 0s - loss: 168.1535

In [33]:
print(f'Part A MSEs Mean and Standard Deviation: {mean_mse}, {std_dev_mse}')
print(f'Part B MSEs Mean and Standard Deviation: {mean_mse_b}, {std_dev_mse_b}')
print(f'Part C MSEs Mean and Standard Deviation: {mean_mse_c}, {std_dev_mse_c}')
print(f'Part D MSEs Mean and Standard Deviation: {mean_mse_d}, {std_dev_mse_d}')

Part A MSEs Mean and Standard Deviation: 70.67206076633134, 51.453265046132685
Part B MSEs Mean and Standard Deviation: 58.523274906805824, 42.89759888951768
Part C MSEs Mean and Standard Deviation: 72.98822520902064, 12.879615696163482
Part D MSEs Mean and Standard Deviation: 39.448376959831876, 14.086881554153406


From the results, we can see that with tripling the # of hidden layers, the model it the most accurate (lowest MSE) with comparable standard deviation. 

(Increasing the # of hidden layers is a more effective method to increase accuracy than just increasing the # of Epochs or iterations.)

<a id="item1"></a>

<a id='item32'></a>

<a id='item33'></a>

<a id="item4"></a>

<a id='item34'></a>